In [ ]:
import os
import sys
from hydromt_sfincs import SfincsModel
import matplotlib.pyplot as plt

!hydromt --version

Create grid from scratch

In [ ]:
# Initialize SfincsModel with the artifact data catalog which contains data for North Italy
sf = SfincsModel(data_libs=["deltares_data"])

In [ ]:
# set output location
sf.set_root("tmp_example", mode="w+")

In [ ]:
# check if your hydromt version has the rotation attribute
from hydromt.raster import RasterDataset
hasattr(RasterDataset, 'rotation')

In [ ]:
inp_dict = {
    "x0": 519106,
    "y0": 5404163,
    "dx": 100.0,
    "dy": 100.0,
    "nmax": 209,
    "mmax": 183,
    "rotation": -37.0148,
    "epsg": 32610,  # crs: "WGS 84 / UTM zone 10N"
}
# create grid
sf.create_grid(grid_type="regular", **inp_dict)
# the input file is automatically updated
sf.region.plot()


Create mask purely based on include- and exclude polygons

In [ ]:
from hydromt_sfincs import utils

In [ ]:
da_dep = sf.data_catalog.get_rasterdataset(
    "gebco", variables=["elevtn"], geom=sf.region, buffer=5
)

# List[dict]
# Provide merge arguments together with xr.DataAraay
da_dep_lst = [{"da":da_dep}]

# Add depth information to modelgrid
sf.create_dep(da_dep_lst=da_dep_lst)

# To plot rotated grids, coordinates must be provided xc and yc
sf.grid["dep"].plot(x='xc', y='yc')

In [ ]:
sf.create_mask_active(elv_min=-5, elv_max=500) #gdf_include=gdf_include, gdf_exclude=gdf_exclude)
sf.mask.plot(x='xc', y='yc')

Also take into account topobathy information

In [ ]:
sf.region.boundary.plot()

In [ ]:
# play around with include, drop_area and fill_area
mask = sf.create_mask_active(
    elv_min=-5, elv_max=500, fill_area=10, drop_area=10, reset_mask=True
)
mask.plot(x='xc',y='yc')

Mask boundaries

In [ ]:
file_name = "tmp_example\\bnd_wl.pol"
gdf_wl = utils.polygon2gdf(feats=utils.read_geoms(fn=file_name), crs=sf.crs)

file_name = "tmp_example\\bnd_out.pol"
gdf_outflow = utils.polygon2gdf(feats=utils.read_geoms(fn=file_name), crs=sf.crs)

In [ ]:
# NOTE that the polygons were created for the non-rotated example
# FIXME: add polygons for this example
sf.create_mask_bounds(btype="waterlevel", gdf_include=gdf_wl, elv_max=0)
sf.create_mask_bounds(btype="outflow", gdf_include=gdf_outflow, elv_min=2)

sf.mask.plot(x='xc',y='yc')

In [ ]:
#reset outflow bounds
sf.create_mask_bounds(btype="outflow", reset_bounds=True)
sf.mask.plot(x='xc',y='yc')

#### subgrid

In [ ]:
sf.create_subgrid(da_dep_lst=da_dep_lst, make_dep_tiles = True)
#sf.create_subgrid(da_dep_lst=da_dep_lst, highres_dir='tmp_example\\tiles')

In [ ]:
sf.subgrid["z_zmin"].plot(x='xc',y='yc')

In [ ]:
sf.subgrid["z_zmax"].plot(x='xc',y='yc')

### write

In [ ]:
sf.write() #write all